In [552]:
import inspect
import re

import numpy as np

from tensorflow import keras  # noqa: F401
from keras import optimizers as tfoptim
from keras import losses as tfloss
from keras import metrics as tfmetric

import torch
import torch.optim as torchoptim
import torch.nn.modules.loss as torchloss
import torchmetrics as torchmetric

import pandas as pd
pd.set_option('display.max_colwidth', 500)

import flatiron.core.tools as fict

In [578]:
def get_classes(module):
    members = inspect.getmembers(module)
    members = list(filter(lambda x: inspect.isclass(x[1]), members))
    members = list(filter(lambda x: not x[0].startswith('_'), members))
    classes = dict(members)
    return classes

def create_signature(arg, annotation, default):
    if annotation == 'UNTYPED':
        annotation = 'Any'
    if default == 'REQUIRED':
        default = ''
    else:
        default = f' = {default}'
    return f'{arg}: {annotation}{default}'

def get_init_signature_data(class_, remove=['self']):
    sig = inspect.getfullargspec(class_)
    args = sig.args
    for item in remove:
        args.remove(item)

    if sig.defaults is not None:
        d = len(args) - len(sig.defaults)
        req = args[:d]
        opt = args[d:]
        args = {k: 'REQUIRED' for k in req}
        opt = dict(zip(opt, sig.defaults))
        args.update(opt)
    else:
        args = {k: 'REQUIRED' for k in args}
    
    if isinstance(sig.kwonlydefaults, dict):
        args.update(sig.kwonlydefaults)
    
    anno = sig.annotations
    for key, val in args.items():
        if key in anno:
            args[key] = (val, anno[key].__name__)
        else:
            args[key] = (val, 'UNTYPED')
            
    data = []
    for arg, (default, type_) in args.items():
        data.append(dict(
            arg=arg,
            default=default,
            type_=type_,
            signature=create_signature(arg, type_, default),
        ))
    return data

def get_module_class_data(module):
    classes = get_classes(module)
    data = []
    for name, item in classes.items():
        try:
            datum = get_init_signature_data(item)
        except:
            continue
        for row in datum:
            row['class_'] = name
        data.extend(datum)
        
    cols = ['class_', 'arg', 'type_', 'default', 'signature']
    data = pd.DataFrame(data, columns=cols)
    data['library'] = module.__name__.split('.')[0]
    data['module'] = module.__name__
    cols.insert(0, 'library')
    cols.insert(1, 'module')
    data = data[cols]
    
    return data

def _get_data(tf_module, torch_module):
    tf_data = get_module_class_data(tf_module)
    torch_data = get_module_class_data(torch_module)
    data = pd.concat([tf_data, torch_data], axis=0)

    mask = data.library == 'keras'
    data.loc[mask, 'library'] = 'tf'
    
    mask = data.library == 'torchmetrics'
    data.loc[mask, 'library'] = 'torch'

    return data

def get_optimizer_data():
    data = _get_data(tfoptim, torchoptim)

    data['field'] = data['class_']
    mask = data.field == 'Nadam'
    data.loc[mask, 'field'] = 'NAdam'
    
    mask = data.class_.apply(lambda x: x not in ['Optimizer', 'LossScaleOptimizer'])
    data = data[mask]
    
    mask = data.arg != 'params'
    data = data[mask]

    data.reset_index(drop=True, inplace=True)

    return data

def get_loss_data():
    data = _get_data(tfloss, torchloss)    
    mask = data.class_.apply(lambda x: x not in ['deprecated'])
    data = data[mask]
    data.reset_index(drop=True, inplace=True)
    return data

def get_metric_data():
    data = _get_data(tfmetric, torchmetric)    
    mask = data.class_.apply(lambda x: x not in ['deprecated'])
    data = data[mask]
    data.reset_index(drop=True, inplace=True)
    return data

def get_class_definitions(data, library, base_class='BaseConfig'):
    caplib = library.capitalize()
    if library == 'tf':
        caplib = 'TF'
    data = data.copy()
    data['config_name'] = data \
        .apply(lambda x: f'class {caplib}{x.class_}({base_class}):', axis=1) \
        .apply(lambda x: re.sub(' Tf', ' TF', x))
    class_def = data \
        .groupby('config_name', as_index=False) \
        .signature.agg(lambda x: '    ' + '\n    '.join(sorted(x)))
    class_def = class_def \
        .apply(lambda x: f'{x.config_name}\n{x.signature}', axis=1) \
        .apply(lambda x: re.sub(' +$', '', x))
    return class_def.tolist()

def get_comparison_data(data, mask=None):
    data = data.copy()
    if mask is not None:
        mask = data.library == mask
        data = data[mask]
    data = data.groupby('arg', as_index=False)[['library', 'class_']].agg(lambda x: x.unique())
    data['len_library'] = data.library.apply(len)
    data['len_class'] = data.class_.apply(len)
    data.sort_values(['len_class', 'len_library'], ascending=False, inplace=True)
    return data

def get_comparison_checkboxes(data, mask=None):
    data = get_comparison_data(data, mask=mask)
    output = data.class_.apply(lambda x: {k: k for k in x}).tolist()
    index = data.arg.tolist()
    output = pd.DataFrame(output, index=index).map(lambda x: '' if pd.isnull(x) else 'x')
    return output

def get_base_class_text(data, arg, class_, base_class, class_re, library):
    mask = data.arg.apply(lambda x: x == arg)
    temp = data[mask]
    if len(temp) == 0:
        raise ValueError(f'{arg} arg not found')
    result = get_class_definitions(temp, library, base_class)[0]
    result = re.sub(class_re, f'class {class_}', result)
    return result

def get_subclass_text(aux, library, class_, inherit, signature, descriptor):
    caplib = library.capitalize()
    if library == 'tf':
        caplib = 'TF'
    inherit = ', '.join(sorted(filter(lambda x: x != '', inherit)))
    output = f'class {caplib}{descriptor}{class_}({caplib}BaseConfig, {inherit}):\n    '
    output = re.sub(r', \)', ')', output)
    regex = '|'.join(aux.keys())
    regex = f'({regex}):'
    if isinstance(signature, str):
        signature = [signature]
    signature = list(filter(lambda x: not re.search(regex, x), signature))
    if signature == []:
        output += 'pass'
    else:
        output += '\n    '.join(sorted(signature))
    output = re.sub(' +$', '', output)
    return output

def print_config_definitions(data, aux, library, descriptor):
    caplib = library.capitalize()
    if library == 'tf':
        caplib = 'TF'
    class_re = f'class {caplib}[a-zA-Z]*'

    mask = data.library == library
    data = data[mask]

    # base class
    base = get_class_definitions(data, library)[0].split('\n')[0]
    base = re.sub(class_re, f'class {caplib}BaseConfig', base)
    base += '\n    name: str'
    print(f'# {library.upper()}' + '-' * 70)
    print(base, '\n\n')

    # helper classes
    print('# HELPERS' + '-' * 70)
    data['inherit'] = ''
    for arg, cls_ in aux.items():
        text = get_base_class_text(data, arg, cls_, 'pyd.BaseModel', class_re, library)
        print(text, '\n\n')
        mask = data.arg == arg
        data.loc[mask, 'inherit'] = cls_

    # subclasses
    print('# ' + '-' * 78)
    class_def = data \
        .sort_values('class_') \
        .groupby('class_', as_index=False)[['inherit', 'signature']] \
        .agg(lambda x: x) \
        .apply(lambda x: get_subclass_text(
            aux, library, x.class_, x.inherit, x.signature, descriptor), axis=1
        ).tolist()
    for item in class_def:
        print(item, '\n\n')

print_tf_metric()

# TF----------------------------------------------------------------------
class TFBaseConfig(BaseConfig):
    name: str 


# HELPERS----------------------------------------------------------------------
class UNIVERSAL(pyd.BaseModel):
    dtype: Any = None 


class UNIVERSAL(pyd.BaseModel):
    name: Any = None 


class TFAxis(pyd.BaseModel):
    axis: Any = -1 


class TFThresh(pyd.BaseModel):
    thresholds: Any = None 


class TFClsId(pyd.BaseModel):
    class_id: Any = None 


class TFNumThresh(pyd.BaseModel):
    num_thresholds: Any = 200 


# ------------------------------------------------------------------------------
class TFMetricAUC(TFBaseConfig, TFNumThresh, TFThresh, UNIVERSAL, UNIVERSAL):
    curve: Any = ROC
    from_logits: Any = False
    label_weights: Any = None
    multi_label: Any = False
    num_labels: Any = None
    summation_method: Any = interpolation 


class TFMetricAccuracy(TFBaseConfig, UNIVERSAL, UNIVERSAL):
    pass 


class TFMetricBinaryAccuracy(TFBas

/tmp/ipykernel_1769482/934884955.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['inherit'] = ''


In [566]:
def print_tf_optimizer_config_definitions():
    # TF
    d = get_optimizer_data()
    mask = d.library == 'tf'
    d = d[mask]

    # TFBaseConfig
    tf_base_args = [
        'clipnorm',
        'clipvalue',
        'ema_momentum',
        'ema_overwrite_frequency',
        'global_clipnorm',
        'gradient_accumulation_steps',
        'learning_rate',
        'loss_scale_factor',
        'name',
        'use_ema',
        'weight_decay',
    ]
    mask = d.arg.apply(lambda x: x in tf_base_args)
    d0 = d[mask]
    tf_base = get_class_definitions(d0)[0]
    tf_base = re.sub('TFAdadeltaConfig', 'TFBaseConfig', tf_base)
    print(tf_base, '\n\n')

    # TFEpsilonBaseConfig
    tf_eps_args = ['epsilon']
    mask = d.arg.apply(lambda x: x in tf_eps_args)
    d1 = d[mask]
    tf_eps = get_class_definitions(d1, 'TFBaseConfig')[0]
    tf_eps = re.sub('class TF[a-zA-Z]*Config', 'class TFEpsilonBaseConfig', tf_eps)
    print(tf_eps, '\n\n')
    
    # TFEpsilonBaseConfig
    tf_eps_args = ['epsilon']
    mask = d.arg.apply(lambda x: x in tf_eps_args)
    d1 = d[mask]
    tf_eps = get_class_definitions(d1, 'TFBaseConfig')[0]
    tf_eps = re.sub('class TF[a-zA-Z]*Config', 'class TFEpsilonBaseConfig', tf_eps)
    print(tf_eps, '\n\n')

    # TFBaseConfig subclasses
    eps_classes = d1.class_.unique().tolist()
    mask = d.class_.apply(lambda x: x in eps_classes)
    d2 = d[~mask]
    mask = d2.arg.apply(lambda x: x in tf_base_args)
    d2 = d2[~mask]
    tf_subclass = get_class_definitions(d2, 'TFBaseConfig')
    for item in tf_subclass:
        print(item, '\n\n')

    # TFEpsilonBaseConfig subclasses
    eps_classes = d1.class_.unique().tolist()
    mask = d.class_.apply(lambda x: x in eps_classes)
    d2 = d[mask]
    mask = d2.arg.apply(lambda x: x in tf_base_args or x in tf_eps_args)
    d2 = d2[~mask]
    tf_eps_subclass = get_class_definitions(d2, 'TFEpsilonBaseConfig')
    for item in tf_eps_subclass:
        print(item, '\n\n')

In [591]:
# tfmetric.OneHotIoU?
# tfmetric.OneHotMeanIoU?
tfmetric.Precision?
tfmetric.PrecisionAtRecall?
tfmetric.R2Score?
# tfmetric.Recall?
# tfmetric.RecallAtPrecision?
tfmetric.SensitivityAtSpecificity?
tfmetric.BinaryIoU?

Init signature:
tfmetric.BinaryIoU(
    target_class_ids=(0, 1),
    threshold=0.5,
    name=None,
    dtype=None,
)
Docstring:     
Computes the Intersection-Over-Union metric for class 0 and/or 1.

Formula:

```python
iou = true_positives / (true_positives + false_positives + false_negatives)
```
Intersection-Over-Union is a common evaluation metric for semantic image
segmentation.

To compute IoUs, the predictions are accumulated in a confusion matrix,
weighted by `sample_weight` and the metric is then calculated from it.

If `sample_weight` is `None`, weights default to 1.
Use `sample_weight` of 0 to mask values.

This class can be used to compute IoUs for a binary classification task
where the predictions are provided as logits. First a `threshold` is applied
to the predicted values such that those that are below the `threshold` are
converted to class 0 and those that are above the `threshold` are converted
to class 1.

IoUs for classes 0 and 1 are then computed, the mean of IoUs 

In [567]:
# def print_tf_loss():
#     data = get_loss_data()
#     aux = dict(
#         reduction='TReduct',
#         reduce='TRed',
#         size_average='TSize',
#         margin='TMarg',
#         weight='TWeight',
#         eps='TEps',
#     )
#     print_config_definitions(data, aux, 'tf', 'Loss')

# def print_tf_optimizer():
#     data = get_optimizer_data()
#     aux = dict(
#         lr='TLR',
#         maximize='TMax',
#         foreach='TFor',
#         differentiable='TDiff',
#         eps='TEps',
#         capturable='TCap',
#         weight_decay='TDecay',
#     )
#     print_config_definitions(data, aux, 'tf', 'Opt')
    
def print_tf_metric():
    data = get_metric_data()
    aux = dict(
        dtype='UNIVERSAL',
        name='UNIVERSAL',
        axis='TFAxis',
        thresholds='TFThresh',
        class_id='TFClsId',
        num_thresholds='TFNumThresh',
    )
    print_config_definitions(data, aux, 'tf', 'Metric')

In [568]:
def print_torch_loss():
    data = get_loss_data()
    aux = dict(
        reduction='TReduct',
        reduce='TRed',
        size_average='TSize',
        margin='TMarg',
        weight='TWeight',
        eps='TEps',
    )
    print_config_definitions(data, aux, 'torch', 'Loss')

def print_torch_optimizer():
    data = get_optimizer_data()
    aux = dict(
        lr='TLR',
        maximize='TMax',
        foreach='TFor',
        differentiable='TDiff',
        eps='TEps',
        capturable='TCap',
        weight_decay='TDecay',
    )
    print_config_definitions(data, aux, 'torch', 'Opt')
    
def print_torch_metric():
    data = get_metric_data()
    aux = dict(
        ignore_index='TInd',
        num_outputs='TOut',
        top_k='TTopK',
    )
    print_config_definitions(data, aux, 'torch', 'Metric')

In [569]:
# print_torch_optimizer()
# print_torch_loss()
# print_torch_metric()

In [540]:
data = get_metric_data()
# data = data[data.class_.apply(lambda x: x in clss)]
# data.groupby('class_').arg.unique()
get_comparison_checkboxes(data, 'tf').T
get_comparison_data(data, 'tf')

,arg,library,class_,len_library,len_class
6,dtype,[tf],"[AUC, Accuracy, BinaryAccuracy, BinaryCrossentropy, BinaryIoU, CategoricalAccuracy, CategoricalCrossentropy, CategoricalHinge, ConcordanceCorrelation, CosineSimilarity, F1Score, FBetaScore, FalseNegatives, FalsePositives, Hinge, IoU, KLDivergence, LogCoshError, Mean, MeanAbsoluteError, MeanAbsolutePercentageError, MeanIoU, MeanMetricWrapper, MeanSquaredError, MeanSquaredLogarithmicError, Metric, OneHotIoU, OneHotMeanIoU, PearsonCorrelation, Poisson, Precision, PrecisionAtRecall, R2Score, Rec...",1,46
15,name,[tf],"[AUC, Accuracy, BinaryAccuracy, BinaryCrossentropy, BinaryIoU, CategoricalAccuracy, CategoricalCrossentropy, CategoricalHinge, ConcordanceCorrelation, CosineSimilarity, F1Score, FBetaScore, FalseNegatives, FalsePositives, Hinge, IoU, KLDivergence, LogCoshError, Mean, MeanAbsoluteError, MeanAbsolutePercentageError, MeanIoU, MeanMetricWrapper, MeanSquaredError, MeanSquaredLogarithmicError, Metric, OneHotIoU, OneHotMeanIoU, PearsonCorrelation, Poisson, Precision, PrecisionAtRecall, R2Score, Rec...",1,46
1,axis,[tf],"[CategoricalCrossentropy, ConcordanceCorrelation, CosineSimilarity, IoU, MeanIoU, OneHotIoU, OneHotMeanIoU, PearsonCorrelation, SparseCategoricalCrossentropy]",1,9
29,thresholds,[tf],"[AUC, FalseNegatives, FalsePositives, Precision, Recall, TrueNegatives, TruePositives]",1,7
4,class_id,[tf],"[Precision, PrecisionAtRecall, Recall, RecallAtPrecision, SensitivityAtSpecificity, SpecificityAtSensitivity]",1,6
19,num_thresholds,[tf],"[AUC, PrecisionAtRecall, RecallAtPrecision, SensitivityAtSpecificity, SpecificityAtSensitivity]",1,5
8,from_logits,[tf],"[AUC, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy]",1,4
10,ignore_class,[tf],"[IoU, MeanIoU, OneHotIoU, OneHotMeanIoU]",1,4
16,num_classes,[tf],"[IoU, MeanIoU, OneHotIoU, OneHotMeanIoU]",1,4
23,sparse_y_pred,[tf],"[IoU, MeanIoU, OneHotIoU, OneHotMeanIoU]",1,4


In [454]:
data = get_metric_data()
x = get_comparison_checkboxes(data, 'torch')
x.map(lambda x: np.nan if x == '' else x).loc['empty_target_action'].dropna()

x = data[data.library == 'torch'].groupby('arg', as_index=False).signature.nunique()
multi_sig_args = x[x.signature > 1].arg.tolist()
sorted(data[data.arg.apply(lambda x: x in multi_sig_args)].signature.tolist())
# .T.map(lambda x: 1 if x == 'x' else 0).sum().head(40)

['empty_target_action: str = neg',
 'empty_target_action: str = neg',
 'empty_target_action: str = neg',
 'empty_target_action: str = neg',
 'empty_target_action: str = neg',
 'empty_target_action: str = neg',
 'empty_target_action: str = neg',
 'empty_target_action: str = neg',
 'empty_target_action: str = neg',
 'empty_target_action: str = pos',
 'kernel_size: Sequence = (11, 11)',
 'kernel_size: Union = 11',
 'kernel_size: Union = 11',
 'lowercase: bool = False',
 'lowercase: bool = False',
 'lowercase: bool = True',
 'metric: Metric',
 'metric: Union',
 'mode: Literal = counts',
 'mode: Literal = speaker-wise',
 'multioutput: Literal = uniform_average',
 'multioutput: str = uniform_average',
 'nan_strategy: Literal = replace',
 'nan_strategy: Literal = replace',
 'nan_strategy: Literal = replace',
 'nan_strategy: Literal = replace',
 'nan_strategy: Union = warn',
 'nan_strategy: Union = warn',
 'nan_strategy: Union = warn',
 'nan_strategy: Union = warn',
 'nan_strategy: Union = war

In [535]:
import flatiron.core.torch_config as fi_torch
fi_torch.TorchMet

<module 'flatiron.core.torch_config' from '/home/ubuntu/flatiron/python/flatiron/core/torch_config.py'>